# Get Instrument Datasheets
### Program written by David Landry
The purpose of this book is to scan project folders, identify instrument datasheets, copy them to a known location if new, override existing file in known location if newer, create an index of spec sheets.

In [1]:
from os import scandir, walk
from pathlib import Path
from datetime import date, datetime
from sys import exit
import shutil
import re
import os
import pandas as pd
import timeit
import logging

instrument_destination_parent = "E:\\OneDrive\\My Programming Projects\\Get Instrument Datasheets\\Go-Bys - Instrument Datasheets"
#instrument_destination_parent = "Go-Bys - Instrument Datasheets"
#instrument_destination_parent = "N:\\users\\dwlandry\\OneDrive\\My Programming Projects\\Get Instrument Datasheets\\Go-Bys - Instrument Datasheets"


current_year = date.today().year
num_of_years_to_search = 5
start_year = current_year - num_of_years_to_search

root_folders_sulphur = []
root_folders_texas = []

for i in range(start_year, current_year + 1):
    folder_sulphur = '//reconshare02/Projects/' + str(i)
    if os.path.isdir(folder_sulphur): root_folders_sulphur.append(folder_sulphur) 
    
    folder_texas = '//reconqnaptx/TX-Projects/Tx Projects/' + str(i)
    if os.path.isdir(folder_texas): root_folders_texas.append(folder_texas) 
    

search_string = '(\.doc|\.xls)'

index = ['client-folder', 'project', 'filename', 'date-modified', 'source-folder']
df = pd.DataFrame()

In [2]:
LOG_FILENAME = instrument_destination_parent + '\\script_log.log'
logging.basicConfig(filename=LOG_FILENAME, filemode='w',level=logging.DEBUG)
logging.info('--------------------------------------------------')
logging.info('RUN DATE: ' + datetime.today().strftime('%Y-%m-%d'))


In [3]:
def flatten_list_of_lists(l):
    #print('Flattening list of lists ...')
    try:
        flat_list = [item for sublist in l for item in sublist]
        return flat_list
    except:
        pass
    
def get_immediate_subdirectories(a_dir):
    #print('Getting immediate subdirectories from', a_dir,'...')
    return [os.path.abspath(os.path.join(a_dir, name)) for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

def get_filepaths(folder, search_string):
    #print('Getting filepaths from', folder, 'using search string (', *search_string,') ...')
    results = []
    for dirpath, dirnames, files in walk(folder):
        for name in files:
            if re.search(search_string, name.lower()):
                #results.append(os.path.join(dirpath, name))
                path = os.path.join(dirpath, name)
                results.append(path)
    return results

def get_client_folders_from_root_folders(root_folders):
    #print('Getting client folders from root folders ...')
    client_folders = []
    for folder in root_folders:
        client_folders.append(get_immediate_subdirectories(folder))
    client_folders = flatten_list_of_lists(client_folders)
    return client_folders

def get_project_folders_from_client_folders(client_folders):
    #print('Getting project folders from client folders ...')
    project_folders = []
    for folder in client_folders:
        project_folders.append(get_immediate_subdirectories(folder))
    project_folders = flatten_list_of_lists(project_folders)
    return project_folders

def get_instrument_files_from_project_folders(project_folders):
    #print ('Getting instrument files from project folders ...')
    instrument_files = []
    
    for project_folder in project_folders:
        instrument_search_folders = [project_folder + '/electrical_instrumentation/instrument engineering/09 - Spec Sheets']
        for folder in instrument_search_folders:
            if os.path.isdir(folder):
                instrument_files.append(get_filepaths(folder, search_string))
    instrument_files = flatten_list_of_lists(instrument_files)
    return instrument_files

def process_instrument_files(instrument_files, int_value_for_project_part_of_path, df):
    #print('Processing instrument files ...')
    project_part = int_value_for_project_part_of_path
    client_part = project_part - 1
    year_part = client_part - 1
    for file in instrument_files:
        p = Path(file)
        project = p.parts[project_part]
        client = p.parts[client_part]
        year = p.parts[year_part]
        folder_name = p.parts[len(p.parts) - 2]
        dst_folder = instrument_destination_parent +'/' + client +'/' + project
        
        src_path = os.path.abspath(file)
        (src_filename, src_ext) = os.path.splitext(p.parts[len(p.parts)-1])
        
        try:
            src_modified = os.stat(src_path).st_mtime
            
            dtetime = datetime.fromtimestamp(src_modified).strftime('%Y-%m-%d-%H%M')
            dst_filename = src_filename + '_' + dtetime
            dst_path = os.path.abspath(dst_folder + '/' + dst_filename + src_ext)
            
            if os.path.exists(dst_path):
                # File already exists.
                # Note that there is no need to compare src_modified time and dst_modifed time since the 
                # datetime is appended to the destinattion filename.
                creation_datetime = datetime.fromtimestamp(os.path.getctime(dst_path)).strftime('%m-%d-%Y')
                modified_datetime = datetime.fromtimestamp(os.path.getmtime(dst_path)).strftime('%m-%d-%Y')
                df = df.append({'YEAR': year,
                                'CLIENT': client,
                                'PROJECT': project,
                                'SOURCE FOLDER': folder_name,
                                'SOURCE PATH': src_path,
                                'DESTINATION FOLDER': dst_folder,
                                'DESTINATION PATH': dst_path,
                                'DATE ADDED': creation_datetime,
                                'DATE MODIFIED': modified_datetime,
                                'STATUS': 'Existing',
                                'FILENAME': src_filename,
                                'FILENAME WITH TIMESTAMP': dst_filename,
                                'FILE EXTENSION': src_ext
                               }, ignore_index=True)
                
            else:
                # New File
                os.makedirs(dst_folder, exist_ok=True)
                shutil.copy2(src_path, dst_path)
                
                creation_datetime = datetime.fromtimestamp(os.path.getctime(dst_path)).strftime('%m-%d-%Y')
                modified_datetime = datetime.fromtimestamp(os.path.getmtime(dst_path)).strftime('%m-%d-%Y')
                df = df.append({'YEAR': year,
                                'CLIENT': client,
                                'PROJECT': project,
                                'SOURCE FOLDER': folder_name,
                                'SOURCE PATH': src_path,
                                'DESTINATION FOLDER': dst_folder,
                                'DESTINATION PATH': dst_path,
                                'DATE ADDED': creation_datetime,
                                'DATE MODIFIED': modified_datetime,
                                'STATUS': 'New',
                                'FILENAME': src_filename,
                                'FILENAME WITH TIMESTAMP': dst_filename,
                                'FILE EXTENSION': src_ext
                               }, ignore_index=True)

                
        except:
            pass
        
    return df


In [4]:
logging.info('Getting client_folders_sulphur...')
client_folders_sulphur = get_client_folders_from_root_folders(root_folders_sulphur)
logging.info('Getting client_folders_texas...')
client_folders_texas = get_client_folders_from_root_folders(root_folders_texas)

logging.info('Getting project_folders_sulphur...')
project_folders_sulphur = get_project_folders_from_client_folders(client_folders_sulphur)
logging.info('Getting project_folders_texas...')
project_folders_texas = get_project_folders_from_client_folders(client_folders_texas)

logging.info('Getting instrument_files_sulphur...')
instrument_files_sulphur = get_instrument_files_from_project_folders(project_folders_sulphur)
logging.info('Getting instrument_files_texas...')
instrument_files_texas = get_instrument_files_from_project_folders(project_folders_texas)

project_part_sulphur = 3
project_part_texas = 4

df = process_instrument_files(instrument_files_sulphur, project_part_sulphur, df)
df = process_instrument_files(instrument_files_texas, project_part_texas, df)

#df.to_excel(instrument_destination_parent + '\index_' + datetime.today().strftime('%Y-%m-%d') + '.xlsx')

In [6]:
logging.info('Getting df_new_datasheets...')
print('Getting df_new_datasheets...')
df_new_datasheets = df[df.STATUS == 'New']

print("df_new_datasheets.shape():", df_new_datasheets.shape)

new_files_filepath = instrument_destination_parent + '\\index-new-records_' + datetime.today().strftime('%Y-%m-%d-%H%M') + '.xlsx'
logging.info('Creating a file for new datasheets and saving at ' + new_files_filepath)
print('Creating a file for new datasheets and saving at', new_files_filepath, '...')
df_new_datasheets.to_excel(new_files_filepath)

Getting df_new_datasheets...
df_new_datasheets.shape(): (9, 13)
Creating a file for new datasheets and saving at E:\OneDrive\My Programming Projects\Get Instrument Datasheets\Go-Bys - Instrument Datasheets\index-new-records_2019-11-19-0901.xlsx ...


## Parse through the Datasheets and map using the master mapping program.
Resource: https://realpython.com/openpyxl-excel-spreadsheets-python/
## Have the Spec Sheet Manager and LPO Program automatically load this data when a new file is added to the folder.

In [3]:
import xlrd
import numpy as np
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

def get_df_map():
    #print('Getting dataframe for map file ...')
    #path_to_datasheet_map = 'datasheet_map.xlsx'
    path_to_datasheet_map = "E:\\OneDrive\\My Programming Projects\\Get Instrument Datasheets\\datasheet_map.xlsx"
    try:
        df_map = pd.read_excel(path_to_datasheet_map)
        return df_map
    except:
        logging.error('Unable to create df_map for ' + path_to_datasheet_map)
        exit(0)
    

def find_datasheet_map_for_xls_xlsx_xlsm(path, df_map):
    #print('Searching the map for a match to', path, '...')
    try:
        workbook = xlrd.open_workbook(filename=path)
        worksheet = workbook.sheet_by_index(0) # test the first worksheet in the workbook.  TODO: Test each worksheet in the workbook.
    except:
        #print("Error Opening Workbook in find_datasheet_map_for_xls_xlsx_xlsm")
        return "Error Opening Workbook in find_datasheet_map_for_xls_xlsx_xlsm"
    #path_to_datasheet_map = 'datasheet_map.xlsx'
    #df_map = get_df_map() #pd.read_excel(path_to_datasheet_map)
    
    for i, row in df_map.iterrows():
        item = row['Key Cells']
        key_cells_list = item.strip('{}').split("', ")
        result = datasheet_match(i, key_cells_list, worksheet)
        if result == True:
            #print('Match found at index', i)
            return i
    #print('No match found')
    return "No Match Found"
       
def datasheet_match(map_index, key_cells, worksheet):
    #print('Comparing worksheet', worksheet.name, 'against map index item', map_index, ' for match to key cells...')#, *key_cells) #, sep = ", ")
    for key_cell in key_cells:
        rng, key_value = [x.strip("'").strip() for x in key_cell.split(': ', 1)]

        xy = coordinate_from_string(rng)
        col = column_index_from_string(xy[0])
        row = xy[1]

        if col>worksheet.ncols or row>worksheet.nrows: return False 
        datasheet_cell_value = worksheet.cell_value(row-1, col-1)
        
        if type(datasheet_cell_value) is str:
            key_value = key_value.strip()
            datasheet_cell_value = datasheet_cell_value.strip()
            if key_value != datasheet_cell_value:
                return False
        else:
            return False

    return True


def number_of_sheets_in_workbook(filename):
    #print('Getting the number of sheets in workbook', filename, '...')
    try:
        book = xlrd.open_workbook(filename)
        return len(book.sheets())
    except:
        return np.nan

def get_serv_manuf_model(index, df_map, datasheet_path, map_col_header):
    #print('Getting Service, Manufacturer and Model Number from', datasheet_path, '...')
    cell = df_map.loc[index][map_col_header]
    if isinstance(cell, str):
        try:
            workbook = xlrd.open_workbook(datasheet_path)
            worksheet = workbook.sheet_by_index(0)
            xy = coordinate_from_string(cell)
            col = column_index_from_string(xy[0])
            row = xy[1]
            return worksheet.cell_value(row-1, col-1)
        except:
            return 'ERROR IN get_serv_manuf_model'
        

In [4]:
logging.info('Getting df_map...')
df_map = get_df_map()

logging.info('Getting df_datasheets...')
#df_datasheets = pd.read_excel(new_files_filepath)
df_datasheets = pd.read_excel('E:\\OneDrive\\My Programming Projects\\Get Instrument Datasheets\\Go-Bys - Instrument Datasheets\\index-new-records_2019-11-21-0107.xlsx')

logging.info('Adding FILE EXTENSION column to df_datasheets.')
df_datasheets['FILE EXTENSION'] = df_datasheets['FILE EXTENSION'].apply(lambda x: x.lower())
logging.info('Adding MATCH column to df_datasheets.')
df_datasheets['MATCH'] = np.nan

logging.info('Creating df_excel as a subset of df_datasheets by filtering FILE EXTENSION for .xls, .xlsx, and .xlsm')
df_excel = df_datasheets[df_datasheets['FILE EXTENSION'].isin(['.xls', '.xlsx', '.xlsm'])]
logging.info('Creating df_doc_docx_docm as a subset of df_datasheets by filtering FILE EXTENSION for .doc, .docx, and .docm')
df_doc_docx_docm = df_datasheets[df_datasheets['FILE EXTENSION'].isin(['.doc', '.docx', '.docm'])]

logging.info('Adding SHEET COUNT column to df_datasheets.')
df_datasheets['SHEET COUNT'] = df_excel['DESTINATION PATH'].apply(number_of_sheets_in_workbook)
#df_datasheets['MATCH'] = df_excel['DESTINATION PATH'].apply(find_datasheet_map_for_xls_xlsx_xlsm)
logging.info('Adding MATCH column to df_datasheets.')
df_datasheets['MATCH'] = df_excel.apply(lambda x: find_datasheet_map_for_xls_xlsx_xlsm(x['DESTINATION PATH'], df_map), axis=1 )

logging.info('Creating df_match_is_str by checking to see if df_datasheets[\'MATCH\'] is an instance of str.')
df_match_is_str = df_datasheets[df_datasheets['MATCH'].apply(lambda x: isinstance(x, (str)))]
if df_match_is_str.shape[0] > 0:
    logging.info('Creating df_no_matches by filtering df_datasheets[\'MATCH\'] to \'No Match Found\'.')
    df_no_matches = df_datasheets[df_datasheets['MATCH'] == 'No Match Found']
    df_no_matches.to_excel(instrument_destination_parent + '\\_no matches found\\no-matches_' + datetime.today().strftime('%Y-%m-%d-%H%M') + '.xlsx')

logging.info('Creating df_matches by filtering df_datasheets[\'MATCH\'] for numeric types.')
df_matches = df_datasheets[df_datasheets['MATCH'].astype(str).str.isnumeric()]
df_matches_nonulls = df_matches[df_matches['MATCH'].notnull()]


In [5]:
if df_matches_nonulls.shape[0]>0:
    logging.info('Adding SERVICE DESCRIPTION to df_datasheets.')
    df_datasheets['SERVICE DESCRIPTION'] = df_matches_nonulls.apply(lambda x: get_serv_manuf_model(x['MATCH'], df_map, x['DESTINATION PATH'], 'Service Cell'), axis=1)
    logging.info('Adding MANUFACTURER to df_datasheets.')
    df_datasheets['MANUFACTUER'] = df_matches_nonulls.apply(lambda x: get_serv_manuf_model(x['MATCH'], df_map, x['DESTINATION PATH'], 'Manufacturer Cell'), axis=1)
    logging.info('Adding MODEL NUMBER to df_datasheets.')
    df_datasheets['MODEL NUMBER'] = df_matches_nonulls.apply(lambda x: get_serv_manuf_model(x['MATCH'], df_map, x['DESTINATION PATH'], 'Model Number Cell'), axis=1)

#df_datasheets['MATCH'].apply(lambda x: isinstance(x, (int, float)))
#df_datasheets.to_excel("index.xlsx")

In [6]:
logging.info('Loading existing index.xlsx.')
print('Loading existing index.xlsx...')
df_index = pd.read_excel(instrument_destination_parent + '\index.xlsx')
logging.info('Appending new datasheets to index.')
print('Appending new datasheets to index...')
df_index = df_index.append(df_datasheets, ignore_index = True)
logging.info('Saving index.')
print('Saving index...')
df_index.to_excel(instrument_destination_parent + '\index.xlsx')
logging.info('Complete')
print('Complete')

Loading existing index.xlsx...
Appending new datasheets to index...
Saving index...
Complete
